In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 9.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [2]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
enc = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
def encode_seq(sentences):
    embeddings = enc.encode(sentences)
    return embeddings

2025-07-14 10:25:38.163020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752488738.387671      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752488738.449155      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer

class TextDataset(Dataset):
    def __init__(self, texts, labels, embedder):
        self.texts = texts
        self.labels = labels
        self.embedder = embedder

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        emb = self.embedder.encode(self.texts[idx], convert_to_tensor=True)
        return emb, torch.tensor(self.labels[idx], dtype=torch.long)

class SBERTClassifier(nn.Module):
    def __init__(self, embed_dim=384, hidden_dim=512, n_classes=2, dropout=0.3):
        super().__init__()
        # MLP head
        self.head = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 4),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 4, n_classes)
        )

    def forward(self, embedding):
        return self.head(embedding)

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for emb, label in dataloader:
        emb, label = emb.to(device), label.to(device)
        logits = model(emb)
        loss = criterion(logits, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for emb, label in dataloader:
            emb, label = emb.to(device), label.to(device)
            preds = model(emb).argmax(dim=1)
            correct += (preds == label).sum().item()
            total += label.size(0)
    return correct / total



In [ ]:
# 1) SBERT 임베더 로드 & 동결
embedder = SentenceTransformer("all-MiniLM-L6-v2")
for param in embedder.parameters():
    param.requires_grad = False

# 2) 데이터 준비 (예시)
texts = ["This is human text.", "Generated by AI..."] * 100
labels = [0, 1] * 100
dataset = TextDataset(texts, labels, embedder)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)




In [ ]:
# 3) 모델·옵티마이저·손실함수 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SBERTClassifier().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
# 4) 학습 루프
for epoch in range(1, 11):
    loss = train(model, train_loader, optimizer, criterion, device)
    acc  = evaluate(model, train_loader, device)
    print(f"Epoch {epoch:02d} — loss: {loss:.4f}, acc: {acc:.4f}")